In [ ]:
import os
import capytaine as cpy
import autograd.numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import brute

import wecopttool as wot
wot.set_loglevel('INFO')

## WEC object
We will go through a number of steps to create a `WEC` object, with which we can solve for the optimal control trajectory.
To compose the `WEC` object, we will need to define the mesh, degrees of freedom, mass and hydrostatic properties, linear hydrodynamic coefficients (from a BEM solution), any additional dynamic forces (e.g. PTO force), and constraints (e.g. minimum line tension).  

### Geometry
First we create a surface mesh for the WEC hull, and quickly visualize the cross-sectional shape of the hull.
The AquaHarmonics hull mesh is already included with WecOptTool—take a look at the `geometry.py` module if you're interested in seeing how it was created.
Note that the lower cylindrical section of the hull is open to the sea.

In [ ]:
buoy_props = {
    'Mass': 3747., # kg
    'CG': 0.242, # m
    'MOI': 7484., # kg-m^2
    'f0': 2 * np.pi /3.0842, # rad/s
}

flywheel_props = {
    'MOI': 4.5,  # kg-m^2
    'Max spring torque': 600,  # N-m
    'Coulomb friction': 2.0,  # N-m
    'Viscous friction': 0.02,  # N-m/rad
    'Damping constant': 10.,   # N-m/rad 
}

spring_props = {
    'Max torque': 40, # N-m
    'Max displacement': np.deg2rad(45.0), # rad
}

In [ ]:
hull_geom = wot.geom.WaveBot(r1=3.302/2,
                             r2=2.76/2, 
                             h1=0.578,
                             h2=0.439,
                             scale_factor=1,
                             freeboard=0.5016)  # use standard dimensions
mesh = hull_geom.mesh(mesh_size_factor=0.5)
_ = hull_geom.plot_cross_section()

Next we create a `FloatingBody` object from the mesh in Capytaine, which we will use to calculate the hydrodynamics.
We can visualize a 3D rendering of the mesh now as well.

In [ ]:
pnr_fb = cpy.FloatingBody.from_meshio(mesh, name="Pioneer")
pnr_fb.add_rotation_dof(name='Pitch')
pnr_fb.add_rotation_dof(name='Flywheel', axis=cpy.meshes.geometry.Axis((0, 1, 0), point=(0, 0, 0)))
pnr_fb.center_of_mass = np.array([0., 0., buoy_props['CG']])
pnr_fb.rotation_center = np.zeros(3)
ndof = pnr_fb.nb_dofs
pnr_fb.show_matplotlib()

### Hydrodynamics and hydrostatics

Next we will run the boundary element model (BEM) [Capytaine](https://github.com/capytaine/capytaine) to obtain the hydrodynamic coefficients for the hull.
Before running the BEM solver, we must specify a set of frequencies at which to perform the calculations.
For `WecOptTool`, these must be a regularly spaced frequency array. 
We will also save the results to a file so that they can be used later, in Part 2, without re-running the BEM solver.

In [ ]:
rho = 1025. # kg/m^3
f1 = 0.025 # Hz
nfreq = 25
freq = wot.frequency(f1, nfreq, False) # False -> no zero frequency

file_name = "data/pioneer_bem.nc"
if os.path.exists(file_name):
    bem_data = wot.read_netcdf(file_name)
else:    
    bem_data = wot.run_bem(pnr_fb, freq)
    wot.write_netcdf(file_name, bem_data)

pnr_fb.keep_immersed_part()
k_buoy = pnr_fb.compute_hydrostatic_stiffness(rho=rho)[0, 0]
k_spring = spring_props['Max torque'] / spring_props['Max displacement']

In [ ]:
bem_data.sel(influenced_dof='Flywheel', radiating_dof='Pitch').added_mass[:] = 0.
bem_data.sel(influenced_dof='Flywheel', radiating_dof='Pitch').radiation_damping[:] = 0.
bem_data.sel(influenced_dof='Pitch', radiating_dof='Flywheel').added_mass[:] = 0.
bem_data.sel(influenced_dof='Pitch', radiating_dof='Flywheel').radiation_damping[:] = 0.
bem_data.sel(influenced_dof='Flywheel', radiating_dof='Flywheel').added_mass[:] = 0.
bem_data.sel(influenced_dof='Flywheel', radiating_dof='Flywheel').radiation_damping[:] = 0.

bem_data.sel(influenced_dof='Flywheel').diffraction_force[:] = 0. + 0j
bem_data.sel(influenced_dof='Flywheel').Froude_Krylov_force[:] = 0. + 0j

In [ ]:
bem_data.sel(influenced_dof='Pitch').diffraction_force

In [ ]:
# plots
fig_am, ax_am = plt.subplots(tight_layout=True, sharex=True)
fig_rd, ax_rd = plt.subplots(tight_layout=True, sharex=True)
fig_ex, ax_ex = plt.subplots(tight_layout=True, sharex=True)

# Excitation
np.abs(bem_data.diffraction_force.sel(influenced_dof='Pitch')).plot(
    ax=ax_ex, linestyle='dashed', label='Diffraction force')
np.abs(bem_data.Froude_Krylov_force.sel(influenced_dof='Pitch')).plot(
    ax=ax_ex, linestyle='dashdot', label='Froude-Krylov force')
ex_handles, ex_labels = ax_ex.get_legend_handles_labels()
ax_ex.set_xlabel(f'$\omega$', fontsize=10)
ax_ex.set_title('Wave Excitation Coefficients', fontweight='bold')
fig_ex.legend(ex_handles, ex_labels, loc='center right', frameon=False)

# Added mass
bem_data.added_mass.sel(
    radiating_dof='Pitch', influenced_dof='Pitch').plot(ax=ax_am)
ax_am.set_xlabel(f'$\omega$', fontsize=10)
ax_am.set_title('Added Mass Coefficients', fontweight='bold')

# Radiation damping
bem_data.radiation_damping.sel(
    radiating_dof='Pitch', influenced_dof='Pitch').plot(ax=ax_rd)
ax_rd.set_xlabel(f'$\omega$', fontsize=10)
ax_rd.set_title('Radiation Damping Coefficients', fontweight='bold')

### PTO system
Using both the parametric kinematics (gear ratios) and the motor power loss map, we can now create a PTO object that will be inserted in our WEC object.
Note that the friction and inertia effects will be included as additional forces.
By defining the PTO object in this way, we will be able to more clearly distinguish the power generating forces from other effects (e.g., friction).

In [ ]:
def gear_ratio(k_spring, moi, nat_freq):
    return np.sqrt(k_spring / (moi * nat_freq**2))

n_default = gear_ratio(k_spring, flywheel_props['MOI'], buoy_props['f0'])

# PTO object
name = ["PTO_Pitch",]
kinematics = np.array([[0, 1]],)
pto_ndof = 1
controller = None
impedance = None # TODO: replace power_loss + f_inertia with this?
pto = wot.pto.PTO(pto_ndof,
                  kinematics=kinematics,
                  impedance=impedance,
                  )

In [ ]:
n_default

### Constraints

In [ ]:
torque_peak_max = 50.
permitted_avg_speed = 3300 * 2*np.pi / 60 # rad/s
def const_peak_torque_pto(wec, x_wec, x_opt, waves, nsubsteps=2):
    torque = pto.force(wec, x_wec, x_opt, waves, nsubsteps) / gear_ratio(k_spring, flywheel_props['MOI'], buoy_props['f0'])
    return torque_peak_max - np.abs(torque.flatten())

def const_avg_velocity_pto(wec, x_wec, x_opt, waves, nsubsteps=2):
    pos = wec.vec_to_dofmat(x_wec)
    pto_vel = np.dot(wec.derivative_mat, pos)[:, 1]
    return permitted_avg_speed - np.mean(pto_vel)

# min_line_tension = 100
# def const_min_tension(wec, x_wec, x_opt, waves, nsubsteps=2):
#     total_tension = -1*f_pto_line(wec, x_wec, x_opt, waves, nsubsteps)
#     return total_tension.flatten() - min_line_tension #TODO

def const_peak_spring_angle(wec, x_wec, x_opt, waves): #TODO: replace this with normalized piece-wise function
    pos = wec.vec_to_dofmat(x_wec)
    rel_pos = pos[:, 0] - pos[:, 1] / n_default
    return spring_props['Max displacement'] - np.abs(rel_pos)

def const_init_start(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos = wec.vec_to_dofmat(x_wec)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    pos_td = np.dot(time_matrix, pos)
    return pos_td[0, 0] - pos_td[0, 1]

### Additional forces

In [ ]:
def f_spring(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos = wec.vec_to_dofmat(x_wec)
    rel_pos = (pos[:, 1] - pos[:, 0]) / n_default
    spring = k_spring * n_default**2 * rel_pos
    torque = np.column_stack([-spring, spring])
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    return np.dot(time_matrix, torque)

def f_friction(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos = wec.vec_to_dofmat(x_wec)
    vel = np.dot(wec.derivative_mat, pos)
    rel_vel = vel[:, 1] - vel[:, 0]
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    fric = -1 * (
        np.tanh(rel_vel) * flywheel_props['Coulomb friction']
      + flywheel_props['Viscous friction'] * rel_vel)
    return np.column_stack([np.zeros(len(fric)), np.dot(time_matrix, fric)])

def f_motor(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos = wec.vec_to_dofmat(x_wec)
    vel = np.dot(wec.derivative_mat, pos)
    rel_vel = vel[:, 1] - vel[:, 0]
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    motor_damping = flywheel_props['Damping constant'] * rel_vel
    motor_damping_td = np.dot(time_matrix, motor_damping)
    return np.column_stack([motor_damping_td, -motor_damping_td])

#TODO: do I need this still?
def f_inertia(wec, x_wec, x_opt, waves, nsubsteps=1):
    pos = wec.vec_to_dofmat(x_wec)
    acc = np.dot(wec.derivative2_mat, pos)
    time_matrix = wec.time_mat_nsubsteps(nsubsteps)
    inertia = np.array([-buoy_props['MOI'], flywheel_props['MOI']]) * acc
    f_inertia = np.dot(time_matrix,inertia)
    return f_inertia

### Create object
Finally, we can use all the different components we've developed thus far to construct a `WEC` object:

 * **BEM data** - defines the hydrodynamics of the hull
 * **Inertia and hydrostatics** - rigid-body inertia and hydrostatic stiffness
 * **Constraints** - limitations on the hardware (max power, max torque, etc.) and the constraint to prevent the tether from going slack
 * **Additional forces** - this captures all of the forces on the WEC that are not due to the interaction between the hull and water (PTO, etc.)

In [ ]:
f_add = {
    # 'Spring': f_spring,
    'Friction': f_friction,
    'Motor damping': f_motor,
    # 'Inertia': f_inertia,
    'PTO': pto.force_on_wec,
}

constraints = [
    {'type': 'ineq', 'fun': const_peak_torque_pto},
    {'type': 'ineq', 'fun': const_avg_velocity_pto},
    # {'type': 'ineq', 'fun': const_min_tension},
    {'type': 'ineq', 'fun': const_peak_spring_angle},
    {'type': 'eq', 'fun': const_init_start}
]

In [ ]:
inertia_matrix = np.array([[buoy_props['MOI'], 0.], [0., flywheel_props['MOI']]])
stiffness = np.array([[k_buoy + k_spring*n_default**2, -k_spring*n_default**2],
                      [-k_spring*n_default**2, k_spring*n_default**2]])

wec = wot.WEC.from_bem(bem_data,
                       inertia_matrix=inertia_matrix,
                       hydrostatic_stiffness=stiffness,
                     #   friction=friction,
                       f_add=f_add,
                       constraints=constraints,
                       dof_names=bem_data.influenced_dof.values,
                        )
# wec._inertia_in_forces = True

## Waves
A regular wave will allow us to get a good initial understanding of the optimal control trajectory.
Note that we'll want to choose a wave frequency that is within the frequency array we used to calculate the hydrodynamic data.

In [ ]:
amplitude = .5/2
# wavefreq =  1/5.3
wavefreq =  0.4 # TODO
phase = 0
wavedir = 0
waves = wot.waves.regular_wave(f1, nfreq, wavefreq, amplitude, phase, wavedir)
# spectrum = lambda f: wot.waves.jonswap_spectrum(freq=f, fp=wavefreq, hs=2*amplitude)
# efth = wot.waves.omnidirectional_spectrum(f1, nfreq, spectrum, "JONSWAP")
# waves = wot.waves.long_crested_wave(efth)
# efth.plot(marker='.')

In [ ]:
# fig, ax = plt.subplots()
# (np.abs(waves)/np.max(np.abs(waves))).plot(marker='.', label='Waves')
# Zi_mag = np.abs(Zi.sel(radiating_dof='Pitch', influenced_dof='Pitch').squeeze())
# (Zi_mag/np.max(Zi_mag)).plot(label='Impedance')

# ax.autoscale(axis='x', tight=True)
# ax.set_ylim((0,1))
# ax.set_title('')
# ax.legend()

## Solve
Finally, we solve for the optimal control trajectory.
To ensure the numerical optimization problem is well-scaled, we set [specific scaling factors](https://snl-waterpower.github.io/WecOptTool/theory.html#scaling) for the WEC position (`scale_x_wec`), the PTO force (`scale_x_opt`), and objective function (`scale_obj`).
We will also set the [maximum iteration and objective function tolerance for the optimization solver](https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html).
This step typically takes about 30s.

In [ ]:

# def my_obj_fun(wec, x_wec, x_opt, waves, steps=1):
#     return np.abs(pto.average_power(wec, x_wec, x_opt, waves, steps))

obj_fun = pto.average_power
nstate_opt = 2*nfreq
results = wec.solve(
    waves,
    obj_fun,
    nstate_opt,
    x_wec_0=np.ones(wec.nstate_wec)*0.8,
    x_opt_0=np.ones(nstate_opt),
    scale_x_wec=1,
    scale_x_opt=1e-3,
    scale_obj=1e-2,
)

## Post-process and plotting
Next, we post-process the results to allow us to more easily visualize them in a series of plots.
Note that because the `WEC` and `PTO` objects are distinct (the WEC object really only knows what the force from the PTO is, not how how that force is obtained), we create two separate results objects (in each case, we get results in the frequency domain and time domain).

In [ ]:
wec_fdom, wec_tdom = wec.post_process(results, waves, nsubsteps=5)
pto_fdom, pto_tdom = pto.post_process(wec, results, waves, nsubsteps=5)
force_excitation = (wec_tdom.force.sel(type='Froude_Krylov') + 
                    wec_tdom.force.sel(type='diffraction'))
pto_tdom.power.mean()
print(f'Optimal average power: {results.fun:.2f} W')

In [ ]:
# Fex = wec_fdom.force.sel(type='Froude_Krylov') + wec_fdom.force.sel(type='diffraction').squeeze().sel(influenced_dof='Pitch')
# P_max = np.abs(Fex)**2 / (8*np.real(Zi.squeeze().sel(influenced_dof='Pitch', radiating_dof='Pitch')))
# fig, ax = plt.subplots()
# P_max.plot()
# ax.set_title('$P_{max}=$' + f'{P_max.sum().item()/1e3:.1f} kW')

In [ ]:
wec_tdom.force.sel(influenced_dof='Pitch').plot(hue='type')

In [ ]:
wec_tdom.force.sel(influenced_dof='Flywheel').plot(hue='type')

In [ ]:
fig, ax = plt.subplots(nrows=4, sharex=True, figsize=(12, 12))

# Excitation

# force_excitation.sel(influenced_dof='Pitch').plot(ax=ax[0], color='k', label='Excitation')
ax[0].set_ylabel('Exc. torque [Nm]')
pto_tdom.force.plot(ax=ax[0], label='PTO', color='r')
# wec_tdom.force.sel(influenced_dof='Pitch').sum(dim='type').plot(ax=ax[0], label='Pitch', color='b')
wec_tdom.force.sel(influenced_dof='Flywheel').sum(dim='type').plot(ax=ax[0], label='Flywheel', color='c')
ax[0].set_ylabel('Torque [Nm]', color='red')
ax[0].tick_params(axis='y', labelcolor='red')
ax[0].set_title('')
ax[0].autoscale(enable=True, axis='x', tight=False)
ax[0].legend()

(wec_tdom.pos*180/np.pi).plot(ax=ax[1], hue='influenced_dof')
ax[1].set_ylabel('Position [deg]')

wec_tdom.vel.plot(ax=ax[2], hue='influenced_dof', add_legend=False)

(pto_tdom['mech_power']).plot(ax=ax[3], 
                                  label='Mech. power ($\\bar{P}_{mech}$: ' + f'{pto_tdom.mech_power.mean().item():.0f} W)')
(pto_tdom['power']).plot(ax=ax[3], linestyle='dashed', 
                         label='Elec. power ($\\bar{P}_{elec}$: ' + f'{pto_tdom.power.mean().item():.0f} W)')

ax[3].legend(loc='upper right')
ax[3].set_ylabel('Power [W]')

for axi in ax:
    axi.set_title('')
    axi.grid()
    axi.label_outer()
    axi.autoscale(axis='x', tight=True)

# # fig.savefig('../gfx/PioneerWec_wecopttool_time_histories.pdf')

In [ ]:
wec_tdom.pos[:, 0]